In [9]:
%pip install tqdm msgraph-core azure-identity requests requests-toolbelt pandas

import sys
from typing import Any, Union
from msgraph.core import GraphClient
from azure.identity import InteractiveBrowserCredential
import os
import json

import requests

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ---------------------------------------- 10.5/10.5 MB 7.4 MB/s eta 0:00:00
     ---------------------------------------- 14.6/14.6 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 503.5/503.5 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
def pretty_print_json(j: Union[str, Any]) -> str:
    """
    Pretty print JSON
    """
    if isinstance(j, str):
        j = json.loads(j)
    return json.dumps(j, indent=2, ensure_ascii=False)

def pprint(p):
    print(pretty_print_json(p))

def warn(p):
    print(p, file=sys.stderr)

# class Page:
#     page_info: str = ""

#     def __init__(self, title, document):
#         self.title = title
#         self.document = document

In [3]:
# Const vals
from typing import List, Any, Iterable

TARGET_NOTEBOOK = 'Obsidian Handwrite'
TARGET_SECTION = '660'



In [4]:
# Get the credential and ensure

scopes = ['User.Read', 'Notes.ReadWrite.All', "Notes.Create", "Notes.Read"]

browser_credential = InteractiveBrowserCredential(
    client_id='cce04e4f-2344-4b02-9ae1-c899d07489b7',
    tenant_id='consumers',
    scopes=scopes,
    redirect_uri='http://localhost:8400/',
)

client = GraphClient(credential=browser_credential, scopes=scopes)

me = client.get('/me')
pprint(me.json())


{
  "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users/$entity",
  "displayName": "Zhou Zxilly",
  "surname": "Zhou",
  "givenName": "Zxilly",
  "id": "60861970f0ea6b24",
  "userPrincipalName": "zxilly@outlook.com",
  "businessPhones": [],
  "jobTitle": null,
  "mail": null,
  "mobilePhone": null,
  "officeLocation": null,
  "preferredLanguage": null
}


In [5]:
# Ensure Notebook exists

ret = client.get('/me/onenote/notebooks')
if not ret.ok:
    raise Exception(str(ret.content))

notebooks = ret.json()

# pprint(notebooks)

target_notebook_id = None

for notebook in notebooks["value"]:
    if notebook['displayName'] == TARGET_NOTEBOOK:
        target_notebook_id = notebook['id']
        break

if target_notebook_id is None:
    warn("Could not find notebook {}".format(TARGET_NOTEBOOK))

    # Create the notebook
    ret = client.post('/me/onenote/notebooks',json={
        "displayName": TARGET_NOTEBOOK,
    })
    if not ret.ok:
        raise Exception(str(ret.content))
    notebook = ret.json()
    target_notebook_id = notebook['id']

print("Found notebook {} at {}".format(TARGET_NOTEBOOK, target_notebook_id))


Found notebook Obsidian Handwrite at 0-60861970F0EA6B24!3652


In [13]:
# Ensure Section exists

ret = client.get(f'/me/onenote/notebooks/{target_notebook_id}/sections')
if not ret.ok:
    raise Exception(str(ret.content))

sections = ret.json()

# pprint(sections)

target_section_id = None

for section in sections["value"]:
    if section['displayName'] == TARGET_SECTION:
        target_section_id = section['id']
        break

if target_section_id is None:
    warn("Could not find section {}".format(TARGET_SECTION))

    # Create the section
    ret = client.post(f'/me/onenote/notebooks/{target_notebook_id}/sections',json={
        "displayName": TARGET_SECTION,
    })
    if not ret.ok:
        raise Exception(str(ret.content))
    section = ret.json()
    target_section_id = section['id']

print("Found section {} at {}".format(TARGET_SECTION, target_section_id))



Found section 660 at 0-60861970F0EA6B24!3663


In [14]:
from tqdm.notebook import trange
from requests_toolbelt import MultipartEncoder

template = """
<html lang="zh-CN">
	<head>
		<title>{}</title>
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	</head>
	<body data-absolute-enabled="true" style="font-family:Microsoft YaHei;font-size:11pt">
		<img src="name:image"/>
	</body>
</html>
"""
end = 661

pages_info = []

for i in trange(387, end):
    page_content = template.format(f"Problem {i}")
    with open(f"data/{i}.png", "rb") as f:
        image = f.read()
    post_data = MultipartEncoder(
        fields={
            "Presentation": ("Presentation", page_content, "text/html"),
            "image": ("image", image, "image/png")
        }
    )
    ret = client.post(f"/me/onenote/sections/{target_section_id}/pages",
                      data=post_data, headers={'Content-Type': post_data.content_type})
    if not ret.ok:
        raise Exception(str(ret.content))
    page = ret.json()
    pages_info.append(page)


  0%|          | 0/274 [00:00<?, ?it/s]